# Assignment for segmenting neighborhoods of Toronto, CA

In [63]:
#import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


## In this section, we will scrape and clean the data to produce the correct dataframe

In [64]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

t_wiki = BeautifulSoup(requests.get(url).text,'lxml') #get soup

In [65]:
cols = ['Postcode',"Borough",'Neighbourhood']#create columns
pID = [] #list of postal ids
bID = [] #list of borough names
nID = [] #list of neighbourhoods
#the next line finds the table, then separates the rows
locs = t_wiki.find_all('tbody')[0].find_all('tr')[1:]

#loop over each row and extract the information
for n in locs:
    d = n.find_all('td')
    pID.append(d[0].text)
    bID.append(d[1].text)
    nID.append(d[2].text.strip())

In [66]:
#create a data frame
t_df = pd.DataFrame(columns=cols)
t_df['Postcode'] = pID
t_df['Borough'] = bID
t_df['Neighbourhood'] = nID
    

In [67]:
t_df.shape

(287, 3)

In [68]:
#Clean data frame to specifications
#This code ignores area without a borough assignment
t_df = t_df[t_df['Borough']!= 'Not assigned']

#This code will assign missing neighborhood names with borough names
correct_hoods = [t_df.iloc[i].Neighbourhood if t_df.iloc[i].Neighbourhood != 'Not assigned'\
                         else t_df.iloc[i].Borough for i in range(len(t_df))]
t_df['Neighbourhood'] = correct_hoods

#Finally, this code will group neighbourhoods by postal code and make lists of hoods
t_df = t_df.groupby('Postcode').agg(lambda x: ', '.join(set(x))).reset_index()

In [70]:
print(t_df.shape)

(103, 3)


In [72]:
t_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"West Hill, Guildwood, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [81]:
str(t_df[t_df['Postcode']== 'M9V'].Neighbourhood)

'101    Albion Gardens, Thistletown, South Steeles, Mo...\nName: Neighbourhood, dtype: object'